In [ ]:
import sys
sys.path.append('../')
import pdb
import pycm
from glob import glob
import torch
from torch import nn
import cv2
import os
from PIL import Image
from skimage import io, transform
import matplotlib.pyplot as plt
import seaborn as sn
import matplotlib.image as mpimg
import pandas as pd
from torchvision.datasets.folder import pil_loader
import random
import numpy as np
from torchsummary import summary
import pretrainedmodels
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *
from tensorboard_logger import * 
from dataloader import *
from submission import *
from albumentations import torch as AT
from pycm import *
os.environ['CUDA_VISIBLE_DEVICES']=""
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
st = 'tensorboard --logdir '
for x in range(5):
    st += f'v{x}:318_effnet-b5_f{x}_pord/,'
print(st)

### preprocessing experiments results

In [ ]:
aug = {}
for i in range(11):
    try:
        print(f'/home/ags/Downloads/predictions/aug_{i}_predictions.csv')
        aug[i] = pd.read_csv(f'/home/ags/Downloads/predictions/aug_{i}_predictions.csv')
    except: 
        pass

In [ ]:
!ls /home/ags/Downloads/predictions/

In [ ]:
aug[3].head()

In [ ]:
tpr = {}
ppv = {}
f1 = {}
for i in range(11):
    try:
        cm = ConfusionMatrix(aug[i]['diagnosis'].values, aug[i]['prediction'].values.astype('uint8'))
        tpr[i] = cm.class_stat['TPR']
        ppv[i] = cm.class_stat['PPV']
        f1[i] = cm.class_stat['F1']
    except Exception as e:
        print(e)
    

In [ ]:
print(cm.class_stat['F1'])

In [ ]:
for c in range(5):
    print(c)
    for key in f1.keys():
        print('aug_%d:' % key, f1[key][str(c)])

In [ ]:
for c in range(5):
    print(c)
    for key in tpr.keys():
        print('aug_%d:' % key, tpr[key][str(c)])

In [ ]:
for c in range(5):
    print(c)
    best_tpr = 0
    for key in tpr.keys():
        if tpr[key][str(c)] > best_tpr:
            best_tpr = tpr[key][str(c)]
            aug = key
    print('aug_%d:' % aug, best_tpr)
       

In [ ]:
for c in range(5):
    metric_list = []
    for key in tpr.keys():
        metric_list.append(["aug_%d" % key, tpr[key][str(c)]])
    print("class %d:" % c, sorted(metric_list, key=lambda x : x[1], reverse=True)[:3]) 

In [ ]:
for c in range(5):
    metric_list = []
    for key in ppv.keys():
        metric_list.append(["aug_%d" % key, ppv[key][str(c)]])
    print("class %d:" % c, sorted(metric_list, key=lambda x : x[1], reverse=True)[:3]) 

In [ ]:
for c in range(5):
    metric_list = []
    for key in f1.keys():
        metric_list.append(["aug_%d" % key, f1[key][str(c)]])
    print("class %d:" % c, sorted(metric_list, key=lambda x : x[1], reverse=True)[:3]) 

In [ ]:
for c in range(5):
    print(c)
    best_f1 = 0
    for key in f1.keys():
        if f1[key][str(c)] > best_f1:
            best_f1 = f1[key][str(c)]
            aug = key
    print('aug_%d:' % aug, best_f1)

In [ ]:
def get_uniques(path):
    '''takes predicted csv's and returns unique counts'''
    pred = pd.read_csv(path)
    print(np.unique(pred.diagnosis.values, return_counts=True))

# def get_preds(pred, threshold):
#     '''pred: [nx5], threshold: float'''
#     y1 = pred > threshold
#     y1 = np.clip(y1.astype(int).sum(axis=1) - 1, 0, 4)
#     return y1

def get_preds(arr, threshold, num_cls=5):
    """ takes in thresholded predictions (num_samples, num_cls) and returns (num_samples,)
    [3], arr needs to be a numpy array, NOT torch tensor"""
    arr = arr > threshold
    mask = arr == 0
    # pdb.set_trace()
    return np.clip(np.where(mask.any(1), mask.argmax(1), num_cls) - 1, 0, num_cls - 1)


def predict(X, coef):
    # [0.15, 2.4, ..] -> [0, 2, ..]
    X_p = np.copy(X)
    for i, pred in enumerate(X_p):
        if pred < coef[0]:
            X_p[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            X_p[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            X_p[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            X_p[i] = 3
        else:
            X_p[i] = 4
    return X_p.astype('int')



In [ ]:
get_uniques('../weights/submission713.csv') 
get_uniques('../weights/submission736.csv') 
get_uniques('../weights/submission73.csv') 
get_uniques('../weights/submission785.csv') 
get_uniques('../weights/submission812.csv') 

In [ ]:
pd.read_csv('../weights/submission820.csv')['diagnosis'].value_counts(normalize=True)

### 4 class test

In [ ]:
npy_folder = '../weights/208_efficientnet-b5_f1_ptest2/test_npy/256/'
files = glob(os.path.join(npy_folder, '*.npy'))
preds = {}
for file in files:
    ep = file.split('/')[-1]
    preds[ep] = np.load(file)
#preds.keys()

In [ ]:
y_test = pd.read_csv('../weights/submission820.csv')['diagnosis'].values#.reshape(-1, 1)
y_test1 = pd.read_csv('../weights/submission812.csv')['diagnosis'].values#.reshape(-1, 1)
# y_test1 = pd.read_csv('../weights/168_efficientnet_poma_ensemble.csv')['diagnosis'].values.reshape(-1, 1)  # 0.808
np.unique(y_test, return_counts=True)

In [ ]:
sub = pd.read_csv('../weights/submission820.csv')
sub['pred'] = 0
sub.head()

In [ ]:
sub.query('diagnosis!=0').index.values

In [ ]:
def predict(X, coef):
    # [0.15, 2.4, ..] -> [0, 2, ..]
    X_p = np.copy(X)
    for i, pred in enumerate(X_p):
        if pred < coef[0]:
            X_p[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            X_p[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            X_p[i] = 2
        else:
            X_p[i] = 3
        #elif pred >= coef[2] and pred < coef[3]:
        #    X_p[i] = 3
        #else:
        #    X_p[i] = 4
    return X_p.astype("int")

# pred[sub.query('diagnosis!=0').index.tolist()] = 

In [ ]:
np.unique(sub.query('diagnosis!=0')['diagnosis'].values, return_counts=True)

In [ ]:
for key in sorted(preds.keys()):
    sub['pred'] = predict(preds[key][0], [0.5, 1.5, 2.5]) + 1
    print(key, np.unique(sub.query('diagnosis!=0')['pred'].values, return_counts=True))

In [ ]:
nonzero = sub.query('diagnosis!=0')#['pred'].value_counts()

In [ ]:
cm = ConfusionMatrix(nonzero['diagnosis'].values, nonzero['pred'].values)

In [ ]:
cm.print_matrix()

In [ ]:
cm.class_stat['F1']

### TEST npy analysis 

In [ ]:
#npy_folder = '../weights/15-7_resnext101_32x4d_v0_fold0_bgcpold256/test_npy/'
#npy_folder = '../weights/17-7_resnext101_32x4d_v0_fold0_bgcpcold256reg/test_npy/'
# npy_folder = '../weights/18-7_resnext101_32x16d_fold0_bgccpold/test_npy/'
# npy_folder = '../weights/268_efficientnet-b4_f1_pordcw/test_npy/256/'
# npy_folder = '../weights/318_senet154_f1_pordpt/test_npy/224/'
npy_folder = '../weights/318_effnet-b5_f1_pordcw/test_npy/256/'
# npy_folder = '../weights/248_efficientnet-b6_f1_po6/test_npy/256/'
files = glob(os.path.join(npy_folder, '*.npy'))
preds = {}
for file in files:
    ep = file.split('/')[-1]
    preds[ep] = np.load(file)
len(sorted(preds.keys()))

In [ ]:
y_test = pd.read_csv('../weights/submission829.csv')['diagnosis'].values#.reshape(-1, 1)
y_test1 = pd.read_csv('../weights/submission820.csv')['diagnosis'].values#.reshape(-1, 1)
# y_test2 = pd.read_csv('../weights/submission812.csv')['diagnosis'].values#.reshape(-1, 1)
# y_test1 = pd.read_csv('../weights/168_efficientnet_poma_ensemble.csv')['diagnosis'].values.reshape(-1, 1)  # 0.808
print(np.unique(y_test, return_counts=True))
cm = ConfusionMatrix(y_test, y_test1)
cm.print_matrix()
cohen_kappa_score(y_test, y_test1, weights="quadratic"), accuracy_score(y_test, y_test1)

In [ ]:
for key in sorted(preds.keys(), key=lambda x: int(x.split('.')[0][9:])):
    pred, val_bt = preds[key]# val_bt = validation best threhsold
    th = 0.5
    th = [0.5, 0.5, 0.5, 0.5, 0.6]
    pred = (pred > th).astype('uint8')
    pred1 = get_preds(pred, 5)
    score = cohen_kappa_score(y_test, pred1, weights="quadratic")
    acc = accuracy_score(y_test, pred1)
    print(f'{key} qwk: {score}, acc: {acc}', np.unique(pred1, return_counts=True)[1])

In [ ]:
# key = 'test_ckpt14.npy' # pordcw
key = 'test_ckpt7.npy' # pord
raw_pred = preds[key][0]
# th = 0.5
th = [0.5, 0.5, 0.5, 0.5, 0.5]
pred = (raw_pred > th).astype('uint8')
pred1 = get_preds(pred, 5)
cm = ConfusionMatrix(y_test, pred1.flatten())
cm.print_matrix()
score = cohen_kappa_score(y_test, pred1, weights="quadratic")
print(f'{key} qwk: {score}, acc: {cm.overall_stat["Overall ACC"]}, counts: {np.unique(pred1, return_counts=True)[1]}')
cm.class_stat['F1']

In [ ]:

def compute_score_inv(thresholds, predictions, targets):
#     pdb.set_trace()
    predictions = (predictions > thresholds).astype('uint8')
    preds = get_preds(predictions, 5)
#     score = accuracy_score(targets, preds)
    score = cohen_kappa_score(preds, targets, weights="quadratic") 
    return 1 - score

def optimize(raw_pred, y_train):
    base_th = [0.5, 0.5, 0.5, 0.5, 0.5]
    simplex = scipy.optimize.minimize(
        compute_score_inv,
        base_th,
        args=(raw_pred, y_train),
        method="nelder-mead",
    )
    best_th = simplex["x"]
    return best_th

In [ ]:
best_th = optimize(raw_pred, y_test)
print('best thresholds:', best_th)
pred = (raw_pred > best_th).astype('uint8')
pred1 = get_preds(pred, 5)
cm = ConfusionMatrix(y_test, pred1.flatten())
cm.print_matrix()
score = cohen_kappa_score(y_test, pred1, weights="quadratic")
print(f'{key} qwk: {score}, acc: {cm.overall_stat["Overall ACC"]}', np.unique(pred1, return_counts=True)[1])
cm.class_stat['F1']

The thing is you don't have to use the optimized thresholds, but you can see where the model is struggling to predict the correct labels and so ease the threshold for that particular category

##### create submission.csv

In [ ]:
sub = pd.read_csv('../data/sample_submission.csv')
sub.loc[:, 'diagnosis'] = pred1
sub.head()

In [ ]:
sub_path = os.path.join(npy_folder, key.replace('.npy', 'opt.csv'))
sub.to_csv(sub_path, index=False)

In [ ]:
df1 = pd.read_csv('../weights/318_senet154_f1_pord/test_npy/224/test_ckpt6.csv')

In [ ]:
cm = ConfusionMatrix(sub['diagnosis'].values, df1['diagnosis'].values)
cm.print_matrix()
# score = cohen_kappa_score(y_test, pred1, weights="quadratic")
# print(f'{key} qwk: {score}, acc: {cm.overall_stat["Overall ACC"]}', np.unique(pred1, return_counts=True)[1])

### Binary classifier

In [ ]:
# ensemble = pd.read_csv('../weights/168_efficientnet_poma_ensemble.csv')
ensemble = pd.read_csv('../weights/submission820.csv')
np.unique(ensemble['diagnosis'], return_counts=True)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
test_classifier_preds = np.load('../classifier/weights/17-8_efficientnet-b5_f0_focalsamp/test_npy/test_ckpt29.npy')
test_classifier_preds = sigmoid(test_classifier_preds)
cls_preds = (test_classifier_preds > 0.6).astype('uint8').flatten()
print('total:', np.sum(cls_preds))
print(ensemble.iloc[np.where(cls_preds == 1)]['diagnosis'].value_counts())

In [ ]:
plt.hist(test_classifier_preds );

In [ ]:
diff = ensemble.iloc[np.where(cls_preds == 1)].query('diagnosis!=4')
diff.shape

In [ ]:
#fig = plt.figure(figsize=(25, 16))
IMG_SIZE=300
for class_id in sorted([0, 1, 2, 3]):
    samples = len(diff[diff['diagnosis'] == class_id])
    if samples == 0: continue
    for i, (idx, row) in enumerate(diff.loc[diff['diagnosis'] == class_id].sample(samples, random_state=69).iterrows()):
        path=f"../data/test_images/{row['id_code']}.png"
        image = cv2.imread(path)
        #print(image.shape)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #     image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        #image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , IMG_SIZE/10) ,-4 ,128) # the trick is to add this line
        plt.imshow(image)
        prob = test_classifier_preds[idx]#[row.index.values]
        plt.title('reg pred: %d, cls pred: %f | %s' % (class_id, prob, image.shape) )
        plt.show()

In [ ]:
ensemble.at[np.where(cls_preds == 1)[0].tolist(), 'diagnosis'] = 4

In [ ]:
np.unique(ensemble['diagnosis'], return_counts=True)

In [ ]:
y_test = pd.read_csv('../weights/submission820.csv')['diagnosis'].values.reshape(-1, 1)
cohen_kappa_score(y_test, ensemble['diagnosis'].values, weights="quadratic"), accuracy_score(y_test, ensemble['diagnosis'].values)

In [ ]:
# ensemble.to_csv('../weights/168_efficientnet_poma_ensemble_withcls.csv', index=False)

In [ ]:
ckpt_path = '../weights/168_efficientnet-b5_f1_poma/test_npy/test_ckpt14.npy'
sub_path = ckpt_path.replace('test_npy/', '').replace('.npy', '.csv')
raw_preds = np.load(ckpt_path)[0]
base_thresholds = [0.5, 1.5, 2.5, 3.5]
df_preds = predict(raw_preds, base_thresholds)
print(np.unique(df_preds, return_counts=True))
df = pd.read_csv('../data/sample_submission.csv')
df.loc[:, 'diagnosis'] = df_preds
# df.to_csv(sub_path, index=False)

In [ ]:
df.diagnosis.head()

In [ ]:
pred, th = preds['test_ckpt17.npy']
best_threshold = [0.5, 1.5, 2.5, 3.5]
pred_ = predict(pred, best_threshold)
print(np.unique(pred_, return_counts=True))
pred_ = predict(pred, th)
print(np.unique(pred_, return_counts=True))


#### Train set  

In [ ]:
train_df = pd.read_csv('../data/train.csv')
train_df['diagnosis'].value_counts()

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
raw_train_preds = sigmoid(np.load('../classifier/weights/17-8_efficientnet-b5_f0_focalsamp/train_npy/train_ckpt29.npy'))
cls_preds = (raw_train_preds > 0.6).astype('uint8').flatten()
print('total:', np.sum(cls_preds))

In [ ]:
plt.hist(raw_train_preds);

In [ ]:
print(train_df.iloc[np.where(cls_preds == 1)]['diagnosis'].value_counts())

In [ ]:
# TPs
same = train_df.iloc[np.where(cls_preds == 1)].query('diagnosis == 4') # those cases which the classifier predicts to be `4`, and they are so -> TPs
same.shape

In [ ]:
# FPs
diff = train_df.iloc[np.where(cls_preds == 1)].query('diagnosis != 4') # those cases which the classifier predicts to be `4`, but they aren't so -> FPs
diff.shape

In [ ]:
#fig = plt.figure(figsize=(25, 16))
IMG_SIZE=300
for class_id in sorted([0, 1, 2, 3]):
    samples = len(diff[diff['diagnosis'] == class_id])
    if samples == 0: continue
    for i, (idx, row) in enumerate(diff.loc[diff['diagnosis'] == class_id].sample(samples, random_state=69).iterrows()):
        path=f"../data/all_images/{row['id_code']}"
        image = cv2.imread(path)
        #print(image.shape)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #     image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , 10) ,-4 ,128) # the trick is to add this line
        plt.imshow(image)
        prob = raw_train_preds[idx]#[row.index.values]
        plt.title('actual: %d, cls pred: %f | %s' % (class_id, prob, image.shape) )
        plt.show()

In [ ]:
#fig = plt.figure(figsize=(25, 16))
IMG_SIZE=300
for class_id in sorted([4]):
    samples = len(same[same['diagnosis'] == class_id])
    if samples == 0: continue
    for i, (idx, row) in enumerate(same.loc[same['diagnosis'] == class_id].sample(samples, random_state=69).iterrows()):
        path=f"../data/all_images/{row['id_code']}"
        image = cv2.imread(path)
        #print(image.shape)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #     image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image=cv2.addWeighted (image, 4, cv2.GaussianBlur( image , (0,0) , IMG_SIZE/10) ,-4 ,128) # the trick is to add this line
        plt.imshow(image)
        prob = raw_train_preds[idx]#[row.index.values]
        plt.title('actual: %d, cls pred: %f | %s' % (class_id, prob, image.shape) )
        plt.show()

### Analysis of model on train/val using cm obj files

In [ ]:
train_cm = ConfusionMatrix(file=open('../weights/168_efficientnet-b5_f2_poma/logs/base_cmtrain_16.obj', 'r'))
cms = {}
for i in range(5):
#     val_cm = ConfusionMatrix(file=open(f'../weights/168_efficientnet-b5_f{i}_poma/logs/base_cmval_16.obj', 'r'))
    val_cm = ConfusionMatrix(file=open(f'../weights/318_effnet-b5_f{i}_pord/logs/base_cmval_16.obj', 'r'))
    cms[i] = val_cm
# val_new_cm = ConfusionMatrix(file=open('../weights/168_efficientnet-b5_f1_oma/logs/base_cmval_new_13.obj', 'r'))

In [ ]:
from collections import defaultdict
f1 = defaultdict(int)
for i in range(5):
    cf1 = cms[i].class_stat['F1']
    for j in range(5):
        f1[j] += cf1[j]/5
    

In [ ]:
f1

In [ ]:
val_cm.print_normalized_matrix()

In [ ]:
train_cm.print_matrix()

In [ ]:
val_cm.print_matrix()

Effnet fold1, ckpt10 is my best performing model so far. LB: 0.812, 
Things to notice:
* Val thresholds optimized were used for val metric val_cm.

In [ ]:
val_cm.overall_stat

In [ ]:
np.mean(list(val_cm.class_stat['TPR'].values()))

In [ ]:
val_cm.class_stat['TPR'], val_cm.class_stat['PPV'], val_cm.class_stat['F1'],

In [ ]:
cm = val_cm
cm.matrix

In [ ]:
cm.TP, cm.FP, cm.TN, cm.FN

In [ ]:
plt.bar(cm.FP.keys(), cm.FP.values())
plt.title('FPs');

In [ ]:
plt.bar(cm.FN.keys(), cm.FN.values())
plt.title('FNs');

### Let's plot some mispredictions

In [ ]:
fig = plt.figure(figsize=(25, 16))
IMG_SIZE=512
for class_id in sorted(diff['actual'].unique()):
    samples = len(diff[diff['actual'] == class_id])
    samples = 5 if samples > 5 else samples # make sure there are atleast 5 predictions to plot or go for whatever is available
    for i, (idx, row) in enumerate(diff.loc[diff['actual'] == class_id].sample(samples, random_state=69).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../data/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , IMG_SIZE/10) ,-4 ,128) # the trick is to add this line
        plt.imshow(image, cmap='gray')
        ax.set_title('Actual: %d Pred: %d | %s' % (class_id, row['pred'], idx) )

In [ ]:
#i = 1546 
i = 494
#img = mpimg.imread() # remember the indices of diff are same as df
path = '../data/train_images/' + df.iloc[i]['id_code'] + '.png'
image = cv2.imread(path)
#image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
#|image=cv2.addWeighted (image, 4, cv2.GaussianBlur( image , (0,0) , IMG_SIZE/10) ,-4 ,128) # the trick is to add this line
fig = plt.figure(figsize=(25, 16))
#plt.imshow(image, cmap='gray')
plt.imshow(image, )
plt.title("Actual: %d | Pred: %d" % (df.iloc[i]['actual'], df.iloc[i]['pred']))
plt.show()

#####  https://www.eyeops.com/contents/our-services/eye-diseases/diabetic-retinopathy

In [ ]:
image = cv2.resize(image, (IMG_SIZE, IMG_SIZE, 3))


### Base model Activation heatmaps

In [ ]:
# Visualize feature maps
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


In [ ]:
model_name = "densenet121"
model = Model(model_name, 5)

In [ ]:
model.model.features.norm5.register_forward_hook(get_activation('norm5'))

In [ ]:
fold =0
total_folds = 10
images_folder = '../data/train_images/'
df_path = '../data/train.csv'
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
dataloaders = {
    phase: provider(
        fold,
        total_folds,
        images_folder,
        df_path,
        phase,
        size,
        mean,
        std,
        batch_size=1,
        num_workers=1,
    )
    for phase in ["train", "val"]
}

def unnormalize(img):
    mean = torch.as_tensor([0.485, 0.456, 0.406])
    std = torch.as_tensor([0.229, 0.224, 0.225])
    #return (img * std) + mean
    for t, m, s in zip(img, mean, std):    
        t.mul_(s).add_(m)
    return img.permute(1, 2, 0)# * 255

In [ ]:
batch = next(iter(dataloaders['train']))

In [ ]:
fnames, images, labels = batch
output = model(images) # during forward pass, model populates the activation dictionary using forward hook

In [ ]:
act = activation['norm5'] # 1, 1024, 7, 7
mean_act = torch.mean(act[0], 0) # take mean across 1024 channels

In [ ]:
plt.imshow(mean_act, cmap='viridis')

In [ ]:
unimage = unnormalize(images[0].clone())

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(unimage);
plt.subplot(1, 2, 2)
plt.imshow(mpimg.imread(f'../data/train_images/{fnames[0]}.png')); 
plt.show(); # original image

In [ ]:
torch.max(unimage), torch.min(unimage)

In [ ]:
torch.max(images), torch.min(images)

### Where's model failing? Hard examples? Analysing train set

In [ ]:
weights_folder = '../weights/318_effnet-b5_f1_pordcw/'
npy_folder = weights_folder + 'train_npy/256/'
files = glob(os.path.join(npy_folder, '*.npy'))
preds = {}
for file in files:
    ep = file.split('/')[-1]
    preds[ep] = np.load(file)
len(sorted(preds.keys()))

In [ ]:
y_train = pd.read_csv('../data/train.csv')['diagnosis'].values

In [ ]:
for key in sorted(preds.keys(), key=lambda x: int(x.split('.')[0].split('_')[1][4:])):
    pred, val_bt = preds[key]# val_bt = validation best threhsold
    #best_threshold = get_optimised_threshold(pred)
#     best_threshold = val_bt
#     best_threshold = np.array([0.5, 1.5, 2.5, 3.3])
#     base_threshold = np.array([0.5, 1.5, 2.5, 3.5])
#     print(f'\n{key}: {best_threshold}')
#     print(f'{key}')
#     pred1 = predict(pred, best_threshold)
#     pred2 = predict(pred, base_threshold)
#     score = cohen_kappa_score(y_test, pred1, weights="quadratic")
#     acc = accuracy_score(y_test, pred1)
#     print('best:', np.unique(pred1, return_counts=True)[1])
#     print(f'best score: {score}, acc: {acc}')
#     print('*' * 10)
#     score = cohen_kappa_score(y_train, pred2, weights="quadratic")
#     acc = accuracy_score(y_test, pred2)
#     print('base:', np.unique(pred2, return_counts=True)[1])
#     print(f'base qwk: {score}, acc: {acc}') 
    th = 0.5
    th = best_th
#     th = [0.5, 0.5, 0.5, 0.4, 0.4]
    pred = (pred > th).astype('uint8')
    pred1 = get_preds(pred, 5)
    score = cohen_kappa_score(y_train, pred1, weights="quadratic")
    acc = accuracy_score(y_train, pred1)

    print('base:', np.unique(pred1, return_counts=True)[1])
    print(f'{key} qwk: {score}, acc: {acc}')


It's utmost important to analyse the predictions on train and val set seperately

In [ ]:
# npy files created with test.py

# train_pred, best_thresholds = np.load('../weights/21-7_efficientnet-b5_fold1_bgccpo300aug2/train_npy/train_ckpt15.npy')
#train_pred, best_thresholds = np.load('../weights/24-7_efficientnet-b5_fold1_poscwwd/train_npy/train_ckpt29.npy')
# train_pred, best_thresholds = np.load('../weights/268_efficientnet-b4_f1_pord/train_npy/256/train_ckpt30.npy')
key = 'train_ckpt15.npy'
raw_pred, best_th = preds[key]
base_th = [0.5, 0.5, 0.5, 0.5, 0.5]
# train_bestpred = predict(train_pred, best_thresholds)
# train_basepred = predict(train_pred, base_th)
train_pred = (raw_pred > base_th).astype('uint8')
train_basepred = get_preds(train_pred, 5)

In [ ]:
# train_df = pd.read_csv('../data/train.csv')
# train_df['diagnosis'] = train_basepred
# train_df.to_csv('../weights/118_efficientnet-b5_f1_postraug2/train_ckpt16.csv', index=False)

In [ ]:
df = pd.read_csv('../data/train.csv')
val_df = pd.read_csv(weights_folder + 'val.csv')
df['is_val'] = df.apply(lambda x: int(x[0] in val_df.id_code.tolist()), axis=1)
df['basepred'] = train_basepred
df['dist'] = abs(df['basepred']-df['diagnosis'])
val_df = df.query('is_val==1')
train_df = df.query('is_val==0')
y_train = df.diagnosis.values

In [ ]:
best_th = optimize(raw_pred, y_train.flatten())
print('best thresholds:', best_th)
pred = (raw_pred > best_th).astype('uint8')
pred1 = get_preds(pred, 5)
cm = ConfusionMatrix(y_train, pred1.flatten())
cm.print_matrix()
score = cohen_kappa_score(y_train, pred1, weights="quadratic")
print(f'{key} qwk: {score}, acc: {cm.overall_stat["Overall ACC"]}', np.unique(pred1, return_counts=True)[1])
cm.class_stat['F1']

In [ ]:
# val set optimization
val_raw_pred = raw_pred[val_df.index]
y_train = val_df.diagnosis.values
best_th = optimize(val_raw_pred, y_train.flatten())
print('best thresholds:', best_th)
pred = (val_raw_pred > best_th).astype('uint8')
pred1 = get_preds(pred, 5)
cm = ConfusionMatrix(y_train, pred1.flatten())
cm.print_matrix()
score = cohen_kappa_score(y_train, pred1, weights="quadratic")
print(f'{key} qwk: {score}, acc: {cm.overall_stat["Overall ACC"]}', np.unique(pred1, return_counts=True)[1])
cm.class_stat['F1']

In [ ]:
# only train set optimization
train_raw_pred = raw_pred[train_df.index]
y_train = train_df.diagnosis.values
best_th = optimize(train_raw_pred, y_train.flatten())
print('best thresholds:', best_th)
pred = (train_raw_pred > best_th).astype('uint8')
pred1 = get_preds(pred, 5)
cm = ConfusionMatrix(y_train, pred1.flatten())
cm.print_matrix()
score = cohen_kappa_score(y_train, pred1, weights="quadratic")
print(f'{key} qwk: {score}, acc: {cm.overall_stat["Overall ACC"]}', np.unique(pred1, return_counts=True)[1])
cm.class_stat['F1']

##### messidor preds

In [ ]:
# messidor train preds
df = pd.read_csv('../data/train_messidor.csv')
key = 'train_ckpt17.npy'
train_pred = np.load('../weights/318_effnet-b5_f1_pord/mes_npy/256/mes_ckpt17.npy')[0]
train_pred = (train_pred > 0.5).astype('uint8')
train_basepred = get_preds(train_pred, 5)
df['basepred'] = train_basepred
df['dist'] = abs(df['basepred']-df['diagnosis'])

In [ ]:
# idrid train preds
df = pd.read_csv('../data/idrid.csv')
key = 'train_ckpt17.npy'
train_pred = np.load('../weights/318_effnet-b5_f1_pord/idrid_npy/256/idrid_ckpt17.npy')[0]
train_pred = (train_pred > 0.5).astype('uint8')
train_basepred = get_preds(train_pred, 5)
df['basepred'] = train_basepred
df['dist'] = abs(df['basepred']-df['diagnosis'])

In [ ]:
df.head()

##### messidor train evaluation

In [ ]:
df.diagnosis.value_counts()
# train_df.head()

In [ ]:
cm = ConfusionMatrix(df['diagnosis'].values, df['basepred'].values)
# cm.print_matrix()
cm.print_normalized_matrix()

IMP: Predictions on messidor, by the model trained on old + finetuned on new only dataset: Almost all class 1 are predicted as 0, overall all heavily biased towards class 0, class 3 predicted as 2, no class 3 and 4 preds (given no class 3 of mess includes 3 and 4 of new dataset)

Re: idrid: it does look like decent pred, except ~70% of class 1 go for class 0, unlike in 2015 where only 10% go for class 0 and ~50% go for class 2

##### Train/Val

In [ ]:
cm = ConfusionMatrix(train_df['diagnosis'].values, train_df['basepred'].values)
cm.print_normalized_matrix()
cm.print_matrix()
cm.overall_stat['Overall ACC']

In [ ]:
cm = ConfusionMatrix(val_df['diagnosis'].values, val_df['basepred'].values)
cm.print_normalized_matrix()
cm.print_matrix()
cm.overall_stat['Overall ACC']

In [ ]:
# print(cm)

In [ ]:
list(cm.class_stat['PPV'].values())

In [ ]:
# qwk in bestpred, basepred
print('qwk:', cohen_kappa_score(train_df['basepred'].values, train_df['diagnosis'].values, weights="quadratic" ))
print('acc:', accuracy_score(train_df['basepred'].values, train_df['diagnosis'].values))

In [ ]:
base_diff = train_df.query("diagnosis != basepred")
# best_diff = train_df.query("diagnosis != bestpred")

In [ ]:
train_df.shape, base_diff.shape, #best_diff.shape # base_diff > best_diff means best_thresholds is better

In [ ]:
diff = base_diff.copy()
diff['pred'] = diff['basepred']
diff = diff.drop(columns=['basepred'])

In [ ]:
print('diff qwk:', cohen_kappa_score(diff['pred'].values, diff['diagnosis'].values, weights="quadratic" ))

In [ ]:
# diff['diagnosis'].value_counts(sort=False).plot('bar');

In [ ]:
# diff['pred'].value_counts(sort=False).plot('bar');

In [ ]:
# class wise mispredictions
# for i in range(5): diff[diff['diagnosis']==i]['pred'].value_counts(sort=False).plot(kind="bar", title=f'Class: {i}'); plt.show()

In [ ]:
cm = ConfusionMatrix(diff['diagnosis'].values, diff['pred'].values)

In [ ]:
cm.print_matrix()

In [ ]:
diff.head()

In [ ]:
# diff.apply(lambda x: print(f'cp all_images/{x[0]} diff/class_{x[1]}'), axis=1);
# diff.to_csv('../data/diff/diff.csv')

In [ ]:
from collections import defaultdict
train_sizes_counts = defaultdict(int) 
train_size2class = defaultdict(list)
for idx, row in tqdm(train_df.iterrows()):
    path = '../data/all_images/' + row.id_code
    img = cv2.imread(path)
    train_sizes_counts[img.shape] += 1
    train_size2class[img.shape].append(row.diagnosis)
train_size2class = {x: np.unique(train_size2class[x], return_counts=True)[1] for x in train_size2class}

In [ ]:
train_sizes_counts

In [ ]:
train_size2class

In [ ]:
from collections import defaultdict
sizes_counts = defaultdict(int) 
size2class = defaultdict(list)
for idx, row in tqdm(diff.iterrows()):
    path = '../data/all_images/' + row.id_code
    img = cv2.imread(path)
    sizes_counts[img.shape] += 1
    size2class[img.shape].append(row.diagnosis)
size2class = {x: np.unique(size2class[x], return_counts=True)[1] for x in size2class}

In [ ]:
size2class 

In [ ]:
sizes_counts

What are these (1736, 2416) and (2136, 3216) images??

In [ ]:
diff.index # indices of hard examples

#### Extremely hard examples

In [ ]:
diff['dist'] = abs(diff['pred'] - diff['diagnosis'])

In [ ]:
diff.head()

In [ ]:
diff['dist'].value_counts(sort=False).plot('bar')

In [ ]:
extremely_hard_examples = diff.query('dist>1')

In [ ]:
extremely_hard_examples.shape

In [ ]:
extremely_hard_examples['diagnosis'].value_counts(sort=False).plot('bar')

In [ ]:
extremely_hard_examples['pred'].value_counts(sort=False).plot('bar')

In [ ]:
excm = ConfusionMatrix(extremely_hard_examples['diagnosis'].values, extremely_hard_examples['pred'].values)
excm.print_matrix()

Extremely hard examples are from class 4, being predicted as 2, If I fix this issue, boi the score is gonna improve

###### Let's visualize these extremely hard examples

In [ ]:
#fig = plt.figure(figsize=(25, 16))
IMG_SIZE=300
# for class_id in sorted(extremely_hard_examples['diagnosis'].unique()):
class_id = 4
samples = len(extremely_hard_examples[extremely_hard_examples['diagnosis'] == class_id])
for i, (idx, row) in enumerate(extremely_hard_examples.loc[extremely_hard_examples['diagnosis'] == class_id].sample(samples, random_state=69).iterrows()):
    path=f"../data/aug_6/{row['id_code']}"
    image = cv2.imread(path)
    print(image.shape)
    plt.imshow(image)
    prob = train_pred[idx]#[row.index.values]
    plt.title('diagnosis: %d pred: %d (%f) | %s' % (class_id, row['pred'], prob, idx) )
    plt.show()

In [ ]:
extremely_hard_examples.index

In [ ]:
extremely_hard_examples.loc[3218]

In [ ]:
#np.save('../data/extremely_hard_examples1.npy', extremely_hard_examples.index.values)

#ehe = np.load('../data/extremely_hard_examples1.npy')


In [ ]:
blur_list = [ 170,  182,  212,  300,  374,  453,  541,  563,  574,  587,  668,
        677,  832,  921, 1029, 1043, 1071, 1075, 1173, 1178, 1233, 1310,
       1341, 1399, 1472, 1485, 1522, 1587, 1600, 1678, 1695, 1798, 1799,
       1800, 1817, 1882, 1979, 1996, 2047, 2054, 2123, 2223, 2319, 2341,
       2392, 2568, 2621, 2625, 2679, 2717, 2751, 2755, 2815, 2842, 2878,
       2924, 2995, 3004, 3024, 3102, 3174, 3196, 3237, 3346, 3375, 3527,
       3553, 3593, 3600, 3644, 3661]

In [ ]:
#ehe = extremely_hard_examples.index.values
train_df.iloc[blur_list].query('dist>0')

In [ ]:
bad_dup_list = np.load('../data/npy_files/bad_train_indices.npy') # the ones which have a duplicate with different diagnosis
print(len(bad_dup_list))
train_df.iloc[dup_list].query('dist>0')

In [ ]:
plt.imshow(np.load('../data/npy_files/bgcc300/4bd941611343.npy')); plt.show()
img = cv2.imread('../data/train_images/4bd941611343.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img); plt.show()

In [ ]:
dup_with_samediag_list = np.load('../data/npy_files/dups_with_same_diagnosis.npy')
print(len(dup_with_samediag_list))
train_df.iloc[dup_with_samediag_list].query('dist>0')

What's the for these getting incorrect predictions? were they in val set?

In [ ]:
plt.imshow(np.load('../data/npy_files/bgcc300/c0e509786f7f.npy'))

In [ ]:
len([x for x in blur_list if x in extremely_hard_examples.index.values])

In [ ]:
#np.save('../data/hard_examples1.npy', best_diff.index.values)

In [ ]:
train_df['diagnosis'].hist()

### Why is there so much variation in the test predictions?

In [ ]:
# let's generate multiple test predictions from a same model

In [ ]:
test0, best_thresholds = np.load('../weights/23-7_efficientnet-b5_fold1_po3005e-6/test_npy/test_ckpt28_0.npy')
test = {
    0: test0
}
for i in range(1, 5):
    test[i] = np.load(f'../weights/23-7_efficientnet-b5_fold1_po3005e-6/test_npy/test_ckpt28_{1}.npy')[0]

In [ ]:
pred = {}
for i in range(5):
    pred[i] = predict(test[i], best_thresholds)

In [ ]:
df = pd.DataFrame()

In [ ]:
for i in range(5):
    df[f'pred{i}'] = pred[i].reshape(-1)

In [ ]:
df.head()

In [ ]:
df.query('pred0 == pred1 == pred2 == pred3 == pred4').shape, df.shape

with tta=2: 1928 - 1824 = 104 cases are varying. 0:55
#with tta=4: 99 cases are varying. 1:22
#with tta=5: 89 cases are varying. 1:22
#with tta=6: 76 cases are varying. 1:48 minutes per epoch
#with tta=6: 94 cases are varying. 1:48 minutes per epoch
#with tta=8: 88 cases are varying. 2:15 minutes per epoch
#with tta=10: 67
#with no tta, got ZERO variation, all predictions were same

### Analyse best submissions

In [ ]:
! ls ../weights/submission*

In [ ]:
# y1 = pd.read_csv('../weights/submission829.csv')
# y2 = pd.read_csv('../weights/submission820.csv')
'''
Remember, y1 and y2 are already ensembles of following submissions.
'''
y3 = pd.read_csv('../weights/submission815.csv')
y4 = pd.read_csv('../weights/submission814.csv')
y5 = pd.read_csv('../weights/submission813.csv')
y6 = pd.read_csv('../weights/submission812.csv')
y7 = pd.read_csv('../weights/submission811.csv')

y = pd.DataFrame()
y['id_code'] = y3['id_code']
# y['y1'] = y1['diagnosis']
# y['y2'] = y2['diagnosis']
y['y3'] = y3['diagnosis']
y['y4'] = y4['diagnosis']
y['y5'] = y5['diagnosis']
y['y6'] = y6['diagnosis']
y['y7'] = y7['diagnosis']

y_columns = [cols for cols in y.columns if 'y' in cols] # get the pred columns
y.head()

In [ ]:
for i in y_columns:
    print(np.unique(y[i].values, return_counts=True))

In [ ]:
from scipy.stats import mode

In [ ]:
a = pd.read_csv('../weights/submission713.csv')
b = pd.read_csv('../weights/submission713.csv')
c = pd.read_csv('../weights/submission713.csv')
majority_voted = (pd.concat([a['diagnosis'], b['diagnosis'], c['diagnosis']], axis=1)).mode(axis=1)

In [ ]:
# combined = pd.concat([a['id_code'], majority_voted],axis=1)
combined = a.copy()

In [ ]:
combined['diagnosis'] = majority_voted
combined.head()

In [ ]:
# calculating standard deviation
y['std'] = np.std(y[y_columns].values.astype('float32'), axis=1)
y['mean'] = np.around(np.mean(y[y_columns].values.astype('float32'), axis=1)).astype('uint')
y['mode'] = y[y_columns].mode(axis=1, numeric_only=True)[0].astype('uint')

print(np.unique(y['mode'].values, return_counts=True))
print(np.unique(y['y3'].values, return_counts=True))

In [ ]:
y.head()

In [ ]:
# histogram of standard deviation
y['std'].plot(kind='hist');

In [ ]:
np.unique(y['std'].values, return_counts=True)

In [ ]:
# analysis shows, ~0.34 values correspond to those which have exactly 1 different predictions
# ~0.45 values correspond to those which have exactly 2 different predictions
# ~0.49 correspond to 3 different
# 0.5 2 different, 2 level apart from mean


In [ ]:
diff = y.query('std>0')
same = y.query('std==0')

In [ ]:
diff.shape, same.shape, y.shape, 100 * diff.shape[0] / y.shape[0] # percentage

In [ ]:
same.head()

In [ ]:
pseudo_test = pd.DataFrame()
pseudo_test['id_code'] = same['id_code']
pseudo_test['diagnosis'] = same['mean']
pseudo_test.to_csv('../data/pseudo_test.csv', index=False)

In [ ]:
diff['std'].plot(kind='hist') # mostly are 1 label apart

In [ ]:
# so `mode` of these predictions is more or less same as 0.829 model which is nonetheless a round(mean) of predictions, wait a min..

In [ ]:
cm = ConfusionMatrix(y['mode'].values, y['y3'].values)
cm.print_matrix()
cohen_kappa_score(y['mode'].values, y['y3'].values, weights="quadratic"), accuracy_score(y['mode'].values, y['y3'].values,)

In [ ]:
cm = ConfusionMatrix(y['mode'].values, y['y4'].values)
cm.print_matrix()
cohen_kappa_score(y['mode'].values, y['y4'].values, weights="quadratic"), accuracy_score(y['mode'].values, y['y4'].values,)

In [ ]:
cm = ConfusionMatrix(y['mode'].values, y['y7'].values)
cm.print_matrix()
cohen_kappa_score(y['mode'].values, y['y7'].values, weights="quadratic"), accuracy_score(y['mode'].values, y['y7'].values,)

In [ ]:

y1 = pd.read_csv('../weights/submission829.csv')
cm = ConfusionMatrix(y['mode'].values, y1['diagnosis'].values)
cm.print_matrix()
cohen_kappa_score(y['mode'].values, y1['diagnosis'].values, weights="quadratic"), accuracy_score(y['mode'].values, y1['diagnosis'].values,)

In [ ]:
sid = ensemble.copy()
sid['b5_loss_preds'] = predict(b5_loss_preds, base_thresholds)
sid['b5_kappa_preds'] = predict(b5_kappa_preds, base_thresholds)
sid['b5_last_preds'] = predict(b5_last_preds, base_thresholds)
sid['b4_loss_preds'] = predict(b4_loss_preds, base_thresholds)
sid['b4_kappa_preds'] = predict(b4_kappa_preds, base_thresholds)
sid['b4_last_preds'] = predict(b4_last_preds, base_thresholds)
sid['b3_loss_preds'] = predict(b3_loss_preds, base_thresholds)
sid['b3_kappa_preds'] = predict(b3_kappa_preds, base_thresholds)
sid['b3_last_preds'] = predict(b3_last_preds, base_thresholds)

### test

In [ ]:
tdf = pd.read_csv('../weights/318_effnet-b5_f0_pord/train.csv')
vdf = pd.read_csv('../weights/318_effnet-b5_f0_pord/val.csv')

# t0 = pd.read_csv('../data/folds/train0-5.csv')
# v0 = pd.read_csv('../data/folds/val0-5.csv')
# t0['id_code'] = t0['id_code'] + '.png'
# v0['id_code'] = v0['id_code'] + '.png'

In [ ]:
sum(tdf.apply(lambda x: x[0] in vdf.id_code.tolist(), axis=1))

In [ ]:
tdf.shape, t0.shape, vdf.shape, v0.shape

In [ ]:
tdf.head()

In [ ]:
tdf[tdf['id_code'] == t0['id_code']].shape

In [ ]:
vdf[vdf['id_code'] == v0['id_code']].shape

In [ ]:
v0[v0['id_code'] == v1['id_code']].shape

### ensemble, train evaluation

In [ ]:
sample_dict = {
        2:    0.647303,
        0:    0.185166,
        1:    0.070539,
        3:    0.058091,
        4:    0.038900
}

count = {}
for i, j in sample_dict.items():
    count[i] = int(sample_dict[i] * 1500) # so that, val size is ~20%

def sample(obj):  # [5]
    return obj.sample(n=count[obj.name], replace=False, random_state=69)

sampled_df = train_df.groupby('diagnosis').apply(sample).reset_index(drop=True)
nonzero = train_df[train_df['diagnosis'] !=0]
sampled_df.shape

In [ ]:
sampled_df.head()

In [ ]:
cohen_kappa_score(sampled_df.diagnosis, sampled_df.basepred, weights="quadratic")

In [ ]:
cohen_kappa_score(nonzero.diagnosis, nonzero.basepred, weights="quadratic")

In [ ]:
diff = train_df.query('diagnosis != basepred')

In [ ]:
diff['pred'] = diff['basepred']
diff = diff.drop(columns=['basepred'])

### test

In [ ]:
import cv2
import numpy as np
import pandas as pd
path = '../data/all_images/b96b518596b3.png'
bgr = cv2.imread(path)
# bgr = crop_image(bgr, tol=10)
bgr = cv2.resize(bgr, (256, 256))
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
k = np.max(rgb.shape)//20*2+1 # kernel size: next odd number to (largest dim of the image) / 10
bg = cv2.medianBlur(rgb, k)
rgb =cv2.addWeighted (rgb, 4, bg, -4, 128)

In [ ]:
plt.imshow(rgb)

In [ ]:
path = '../data/all_images/b96b518596b3.png'
bgr = cv2.imread(path)
bgr = crop_image(bgr, tol=10)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
k = np.min(rgb.shape)//20*2+1 # kernel size: next odd number to (largest dim of the image) / 10
print(k)
k=351
bg = cv2.medianBlur(rgb, k)
rgb =cv2.addWeighted (rgb, 4, bg, -4, 128)
rgb = cv2.resize(rgb, (256, 256))
plt.imshow(rgb)